In [14]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re

import numpy as np

In [15]:
#Function to load data

# Fonction pour extraire le nom de l'expérience à partir du nom de fichier
def extract_experiment_name(file_name, pattern):
    # Expression régulière pour rechercher le motif "P-V 1V_1#1" dans le nom de fichier
    match = re.search(pattern, file_name)
    if match:
        return match.group()
    else:
        return None

def load_data(data_folder, chip_name, placement, size, graph_type):
    # Liste des dossiers dans le dossier data_folder
    date_folders = os.listdir(data_folder)
    
    # Parcours des dossiers de date
    for date_folder in date_folders:
        # Chemin complet du dossier de date
        date_folder_path = os.path.join(data_folder, date_folder)
        
        # # Liste des dossiers de chips dans le dossier de date
        # chip_folders = os.listdir(date_folder_path)

        # Liste des dossiers de chips dans le dossier de date
        chip_folders = [folder for folder in os.listdir(date_folder_path) if os.path.isdir(os.path.join(date_folder_path, folder))]
        
        
        # Parcours des dossiers de chips
        for chip_folder in chip_folders:
            # Vérification si le nom de la chip correspond
            if chip_name in chip_folder:
                # Chemin complet du dossier de la chip
                chip_folder_path = os.path.join(date_folder_path, chip_folder)
                
                # Liste des fichiers xls dans le dossier de la chip
                xls_files = [file for file in os.listdir(chip_folder_path) if file.endswith('.xls')]
                
                # Parcours des fichiers xls
                for xls_file in xls_files:
                    # Séparation du nom du fichier
                    file_parts = xls_file.split('_')
                    file_placement, file_size, end = file_parts[0], file_parts[1], '_'.join(file_parts[4:])
                    file_graph_type = extract_experiment_name(xls_file, graph_type)
                    #file_graph_type = file_graph_type.split('.')[0]
                    
                    # Vérification si les critères de recherche correspondent
                    if file_placement == placement and file_size == size and file_graph_type == graph_type:
                        # Chargement des données xls dans une DataFrame
                        xls_path = os.path.join(chip_folder_path, xls_file)
                        data = pd.read_excel(xls_path)
                        return data

In [24]:
def plot_data(data_list, experience, placement, size, new_chip_list, Size_num, output_type_folder, graph_type):
    plt.figure(figsize=(25, 12.5))
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'orange', 'cyan']  # Liste de couleurs pour les courbes
    area = (Size_num*10**(-6))**2
    for i, chip_name in enumerate(new_chip_list):
        data = data_list[i]
        plt.plot(data['Vforce'], data['Charge']/area, marker='o', label=f"{chip_name} - {graph_type}", 
                        color=colors[i-1], linewidth=0.8)
    plt.title(f'Polarisation vs Voltage ({placement}, Size {size}, experience {graph_type}, Type {experience})', fontsize=25)
    plt.xlabel('Voltage', fontsize=25)
    plt.ylabel('Polarisation', fontsize=25)
    plt.grid(True)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=25)
    plt.tick_params(axis='both', labelsize=20)
    # Sauvegarder le plot dans le dossier spécifié avec le titre comme nom de fichier
    plt.savefig(os.path.join(output_type_folder, f"Polarisation_vs_Voltage_{placement}_{size}_{graph_type}_{experience}.png"))
    plt.close()

def plot_pump(data_list, experience, placement, size, new_chip_list, output_type_folder, graph_type):
    plt.figure(figsize=(25, 12.5))
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'orange', 'cyan']  # Liste de couleurs pour les courbes
    for i, chip_name in enumerate(new_chip_list):
        data = data_list[i]
        plt.plot(data['t'], data['I'], marker='o', label=f"{chip_name} - {graph_type}", 
                        color=colors[i-1], linewidth=0.8)
    plt.title(f'Current vs Time ({placement}, Size {size}, experience {graph_type}, Type {experience})', fontsize=25)
    plt.xlabel('Time', fontsize=25)
    plt.ylabel('Current', fontsize=25)
    plt.grid(True)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=25)
    plt.tick_params(axis='both', labelsize=20)
    # Sauvegarder le plot dans le dossier spécifié avec le titre comme nom de fichier
    plt.savefig(os.path.join(output_type_folder, f"Current_vs_Time_{placement}_{size}_{graph_type}_{experience}.png"))
    plt.close()


def plot_leakage(data_list, experience, placement, size, new_chip_list, output_type_folder, graph_type):
    plt.figure(figsize=(25, 12.5))
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'orange', 'cyan']  # Liste de couleurs pour les courbes
    for i, chip_name in enumerate(new_chip_list):
        data = data_list[i]
        plt.plot(data['AV'], data['AI'], marker='o', label=f"{chip_name} - {graph_type}", 
                        color=colors[i-1], linewidth=0.8)
    plt.title(f'Current vs Voltage ({placement}, Size {size}, experience {graph_type}, Type {experience})', fontsize=25)
    plt.xlabel('Time', fontsize=25)
    plt.ylabel('Current', fontsize=25)
    plt.grid(True)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=25)
    plt.tick_params(axis='both', labelsize=20)
    # Sauvegarder le plot dans le dossier spécifié avec le titre comme nom de fichier
    plt.savefig(os.path.join(output_type_folder, f"Current_vs_voltage_{placement}_{size}_{graph_type}_{experience}.png"))
    plt.close()

In [25]:

type_list = ["DP", "SP", "600", "500", "450", "20", "120", "240"]   
size_list = ["50", "100", "150", "200", "500"]
for experience in type_list:
    for size in size_list:
        placement = "tr"
            
        output_folder = "C:\\Documents\\EPFL\\MA4\\Projet_de_semestre\\Code\\Plot"

        data_list = []
        new_chip_list = []
        data_folder = "C:\Documents\EPFL\MA4\Projet_de_semestre\Code\Data2"
        Size_num = int(size)

        output_type_folder = os.path.join(output_folder, f"{experience}_{size}_{placement}")
        os.makedirs(output_type_folder, exist_ok=True)

        if experience == "DP":
            # 3dec17, 3dec11, 3dec09, dec15
            chips = ["3dec17", "3dec11", "3dec09", "dec15"]
        elif experience == "SP":
            # 3dec16, 3dec08, 3dec10, 3dec02, 3dec14
            chips = ["3dec16", "3dec08", "3dec10", "3dec02", "3dec14"]
        elif experience == "600":
            # 3dec18
            chips = ["3dec18"]
        elif experience == "500":
            # 3dec16, 3dec08, 3dec10, 3dec02, 3dec17, 3dec11
            chips = ["3dec16", "3dec08", "3dec10", "3dec02", "3dec17", "3dec11"]
        elif experience == "450":
            # 3dec14, 3dec15, 3dec09
            chips = ["3dec14", "3dec15", "3dec09"]
        elif experience == "20":
            # 3dec17, 3dec15, 3dec16, 3dec14
            chips = ["3dec17", "3dec15", "3dec16", "3dec14"]
        elif experience == "120":
            # 3dec11, 3dec09,3dec08, 3dec10
            chips = ["3dec11", "3dec09", "3dec08", "3dec10"]
        elif experience == "240":
            # 3dec02
            chips = ["3dec02"]

        # list_exp_1 = ["P-V 1V_1#1", "P-V 1V_2#1", "P-V 2V_1#1", "P-V 2V_2#1", "P-V 3V_1#1", "P-V 3V_2#1", 
        #                     "P-V 4V_1#1", "P-V 4V_2#1", "P-V 3V neg_1#1", "P-V 3V neg_1#1", "PUND 5V_1#1", "PUND 5V neg_1#1"]
        
        list_exp_1 = [ "IV 3V_1#1"]

        # Charger les données pour chaque chip et les stocker dans data_list
        for graph_type in list_exp_1:
            print(len(chips))
            for i, chip_name in enumerate(chips):
                data = load_data(data_folder, chip_name, placement, size, graph_type)
                if data is not None:
                    data_list.append(data)
                    new_chip_list.append(chip_name)
                    print(f"Les données pour la chip {chip_name} et le graph_type {graph_type} ont été chargées avec succès.")
                else:
                    print(f"Aucun fichier correspondant pour la chip {chip_name} et le graph_type {graph_type}.")
                # Tracer les données  

            if len(data_list) != 0:
                if graph_type in ["PUND 5V_1#1", "PUND 5V neg_1#1"]:
                    plot_pump(data_list, experience, placement, size, new_chip_list, output_type_folder, graph_type)
                if graph_type in ["IV 3V_1#1"]:
                    plot_leakage(data_list, experience, placement, size, new_chip_list, output_type_folder, graph_type)
                if graph_type in ["P-V 1V_1#1", "P-V 1V_2#1", "P-V 2V_1#1", "P-V 2V_2#1", "P-V 3V_1#1", "P-V 3V_2#1", 
                            "P-V 4V_1#1", "P-V 4V_2#1", "P-V 3V neg_1#1", "P-V 3V neg_1#1"]:
                    plot_data(data_list, experience, placement, size, new_chip_list, Size_num, output_type_folder, graph_type)  
            new_chip_list.clear()
            data_list.clear()

4
WARNING *** file size (40643) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip 3dec17 et le graph_type IV 3V_1#1 ont été chargées avec succès.
WARNING *** file size (42250) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip 3dec11 et le graph_type IV 3V_1#1 ont été chargées avec succès.
WARNING *** file size (42083) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip 3dec09 et le graph_type IV 3V_1#1 ont été chargées avec succès.
WARNING *** file size (41878) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip dec15 et le graph_type IV 3V_1#1 ont été chargées avec succès.
4
WARNING *** file size (40699) not 512 + multiple of sector size (512)

In [26]:
output_folder = "C:\\Documents\\EPFL\\MA4\\Projet_de_semestre\\Code\\Plot"

data_list = []
new_chip_list = []
data_folder = "C:\Documents\EPFL\MA4\Projet_de_semestre\Code\Data2"

# Demander à l'utilisateur le nombre de chips à charger
experience = input("What did you want to compare DP, SP, 600, 500, 450, 20, 120, 240")
placement = input(f"Entrez le placement pour la chip (tr, tl, br, bl): ")
size = input(f"Entrez la taille pour la chip (50, 100, 150, 200, 500): ")
Size_num = int(size)

output_type_folder = os.path.join(output_folder, f"{experience}_{size}_{placement}")
os.makedirs(output_type_folder, exist_ok=True)

if experience == "DP":
    # 3dec17, 3dec11, 3dec09, dec15
    chips = ["3dec17", "3dec11", "3dec09", "dec15"]
elif experience == "SP":
    # 3dec16, 3dec08, 3dec10, 3dec02, 3dec14
    chips = ["3dec16", "3dec08", "3dec10", "3dec02", "3dec14"]
elif experience == "600":
    # 3dec18
    chips = ["3dec18"]
elif experience == "500":
    # 3dec16, 3dec08, 3dec10, 3dec02, 3dec17, 3dec11
    chips = ["3dec16", "3dec08", "3dec10", "3dec02", "3dec17", "3dec11"]
elif experience == "450":
    # 3dec14, 3dec15, 3dec09
    chips = ["3dec14", "3dec15", "3dec09"]
elif experience == "20":
    # 3dec17, 3dec15, 3dec16, 3dec14
    chips = ["3dec17", "3dec15", "3dec16", "3dec14"]
elif experience == "120":
    # 3dec11, 3dec09,3dec08, 3dec10
    chips = ["3dec11", "3dec09", "3dec08", "3dec10"]
elif experience == "240":
    # 3dec02
    chips = ["3dec02"]

list_exp_1 = ["P-V 1V_1#1", "P-V 1V_2#1", "P-V 2V_1#1", "P-V 2V_2#1", "P-V 3V_1#1", "P-V 3V_2#1", 
                      "P-V 4V_1#1", "P-V 4V_2#1", "P-V 3V neg_1#1", "P-V 3V neg_1#1", "PUND 5V_1#1", "PUND 5V neg_1#1", "IV 3V_1#1"]

# Charger les données pour chaque chip et les stocker dans data_list
for graph_type in list_exp_1:
    print(len(chips))
    for i, chip_name in enumerate(chips):
        data = load_data(data_folder, chip_name, placement, size, graph_type)
        if data is not None:
            data_list.append(data)
            new_chip_list.append(chip_name)
            print(f"Les données pour la chip {chip_name} et le graph_type {graph_type} ont été chargées avec succès.")
        else:
            print(f"Aucun fichier correspondant pour la chip {chip_name} et le graph_type {graph_type}.")
        # Tracer les données  

    if len(data_list) != 0:
        if graph_type in ["PUND 5V_1#1", "PUND 5V neg_1#1"]:
            plot_pump(data_list, experience, placement, size, new_chip_list, output_type_folder, graph_type)
        if graph_type in ["IV 3V_1#1"]:
            plot_leakage(data_list, experience, placement, size, new_chip_list, output_type_folder, graph_type)
        if graph_type in ["P-V 1V_1#1", "P-V 1V_2#1", "P-V 2V_1#1", "P-V 2V_2#1", "P-V 3V_1#1", "P-V 3V_2#1", 
                      "P-V 4V_1#1", "P-V 4V_2#1", "P-V 3V neg_1#1", "P-V 3V neg_1#1"]:
            plot_data(data_list, experience, placement, size, new_chip_list, Size_num, output_type_folder, graph_type)  
    new_chip_list.clear()
    data_list.clear()

4
WARNING *** file size (195732) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip 3dec17 et le graph_type P-V 1V_1#1 ont été chargées avec succès.
WARNING *** file size (195733) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip 3dec11 et le graph_type P-V 1V_1#1 ont été chargées avec succès.
WARNING *** file size (195731) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip 3dec09 et le graph_type P-V 1V_1#1 ont été chargées avec succès.
WARNING *** file size (195731) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Les données pour la chip dec15 et le graph_type P-V 1V_1#1 ont été chargées avec succès.
4
WARNING *** file size (164043) not 512 + multiple of sector s